In [ ]:
!git clone https://github.com/Yasgant/project-nmnbns.git
!cp ./project-nmnbns/game/* ./ -rf
!pip install pygame

Cloning into 'project-nmnbns'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 64 (delta 29), reused 49 (delta 17), pack-reused 0
Unpacking objects: 100% (64/64), done.
     |████████████████████████████████| 21.8 MB 1.1 MB/s 


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import deque
import random
import sys
sys.path.append('/content')
from core import *
from danmaku import *
from enemies import *
from tensorflow.keras import layers, optimizers, models
import copy
from collections import deque

pygame 2.1.2 (SDL 2.0.16, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
from tensorflow.python.util.nest import flatten
model = models.Sequential([
    layers.Flatten(input_shape = (4000,)),
    layers.Dense(81, activation='relu'),
    layers.Dense(9, activation="linear")
])
model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(0.001))

In [ ]:
model2 = models.Sequential([
    layers.Input(shape = (640, 480, 1)),
    layers.Conv2D(filters = 32, kernel_size = (4,4)),
    layers.Flatten(),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(9, activation='linear')
])
model2.compile(loss='mean_squared_error', optimizer=optimizers.Adam(0.001))

In [ ]:
def disp(frame):
    image = np.zeros_like(frame)
    for i in range(len(frame)):
        for j in range(len(frame[0])):
            image[i][j] = 0 if frame[i][j] else 255
    plt.figure()
    plt.title(image.shape)
    plt.imshow(image, cmap='gray', vmin=0, vmax=255)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save('MyModel')
class MyModel:
    def __init__(self):
        self.model = model
        self.target_model = copy.deepcopy(self.model)
        self.update_freq = 200
        self.replay_size = 2000
        self.replay_queue = deque(maxlen = self.replay_size)
        self.step = 0

    def pred(self, frame, epsilon = 0.1):
        if np.random.uniform() < epsilon:
            return np.random.randint(9)
        #print(self.model.predict(frame))
        return np.argmax(self.model.predict(np.array([frame]))[0])
    
    def remember(self, frame, action, next_frame, reward):
        self.replay_queue.append([frame, action, next_frame, reward])
    
    def train(self, batch_size = 256, lr = 0.1, gamma = 0.97):
        if len(self.replay_queue) < 300:
            return
        self.step += 1
        if self.step % self.update_freq == 0:
            self.target_model.set_weights(self.model.get_weights())

        replay_batch = random.sample(self.replay_queue, batch_size)
        frame_batch = np.array([r[0] for r in replay_batch])
        next_frame_batch = np.array([r[2] for r in replay_batch])
        Q = self.model.predict(frame_batch)
        Q_next = self.target_model.predict(next_frame_batch)

        for i, replay in enumerate(replay_batch):
            _, action, _, reward = replay
            Q[i][action] = (1 - lr) * Q[i][action] + lr * (reward + gamma * np.amax(Q_next[i]))
        self.model.fit(frame_batch, Q, verbose = 0)



data2 = StageData.empty('S1',10*60*60)
# 0 ~ 1000
data2.insert(0, AimmingEnemy3(100, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(9, AimmingEnemy3(110, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(18, AimmingEnemy3(120, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(27, AimmingEnemy3(130, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(36, AimmingEnemy3(140, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(45, AimmingEnemy3(150, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(54, AimmingEnemy3(160, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(63, AimmingEnemy3(170, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(72, AimmingEnemy3(180, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))
data2.insert(81, AimmingEnemy3(190, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=10*60))

# 1000 ~ 1500
data2.insert(1000, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1020, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1040, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1060, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1080, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1100, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1000, AimmingEnemy(540, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1020, AimmingEnemy(540, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1040, AimmingEnemy(540, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1060, AimmingEnemy(540, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1080, AimmingEnemy(540, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data2.insert(1100, AimmingEnemy(540, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))

# 1500 ~ 2500
data2.insert(1500, RandomEnemy(320, 100, shoot_time = 5, bullet_speed=5, alive_time=7*60))

# 2500 ~ 3500
data2.insert(2500, RandomEnemyWithFall(320, 100, shoot_time = 5, bullet_speed=5, alive_time=7*60))


episodes = 1000  
score_list = [] 
agent = MyModel()
ccc = -1
cnt = 0
for i in range(episodes):
    ccc += 1
    G = game_with_op(Player(320, 400), copy.deepcopy(data2))
    score = 0
    with open(str(ccc)+'.txt', 'w') as f:
        preframe = G.get_enemies()
        while True:
            frame = G.get_enemies()
            if not (any(frame[1:1000]) or any(frame[1001: 2000])):
                _, don = G.op(0)
                print('skipping')
                if don:
                    raise Except('???')
                f.write(str(0)+'\n')
                continue
            frame.extend(preframe)
            action = agent.pred(frame)
            reward, done = G.op(action)
            next_frame = G.get_enemies()
            next_frame.extend(frame[:2000])
            agent.remember(frame, action, next_frame, reward)
            #if cnt % 20 == 0:
            agent.train()
            cnt += 1
            score += reward
            f.write(str(action)+'\n')
            #disp(G.centered_fill())
            print(G.frame, score, action)
            print('x:',G.player.x,'y:', G.player.y)
            preframe = frame[:2000]
            if done:
                score_list.append(G.frame)
                print('episode:', i, 'score:', score, 'frame:', G.frame, 'max:', max(score_list), np.argmax(score_list))
                break

import matplotlib.pyplot as plt

plt.plot(score_list, color='green')
plt.show()
    

INFO:tensorflow:Assets written to: MyModel/assets
INFO:tensorflow:Assets written to: ram://7b97780d-9a6a-4c5e-8c83-e4ab7882edaf/assets
skipping
2 200 1
x: 318 y: 400
3 400 1
x: 316 y: 400
4 600 1
x: 314 y: 400
5 800 1
x: 312 y: 400
6 1000 1
x: 310 y: 400
7 1200 2
x: 308.6 y: 401.4
8 1400 1
x: 306.6 y: 401.4
9 1600 1
x: 304.6 y: 401.4
10 1800 1
x: 302.6 y: 401.4
11 2000 6
x: 304.0 y: 400.0
12 2200 4
x: 305.4 y: 401.4
13 2400 6
x: 306.79999999999995 y: 400.0
14 2600 6
x: 308.19999999999993 y: 398.6
15 2800 6
x: 309.5999999999999 y: 397.20000000000005
16 3000 1
x: 307.5999999999999 y: 397.20000000000005
17 3200 6
x: 308.9999999999999 y: 395.80000000000007
18 3400 5
x: 310.9999999999999 y: 395.80000000000007
19 3600 6
x: 312.39999999999986 y: 394.4000000000001
20 3800 6
x: 313.79999999999984 y: 393.0000000000001
21 4000 6
x: 315.1999999999998 y: 391.60000000000014
22 4200 6
x: 316.5999999999998 y: 390.20000000000016
23 4400 6
x: 317.9999999999998 y: 388.8000000000002
24 4600 6
x: 319.39999

KeyboardInterrupt: ignored